In [33]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import hopsworks
import json
import os

In [34]:
import requests
import csv

# Replace this with your Clash Royale API token
key_file_path = os.path.join('../data/clash-royale-api-key.txt')
with open(key_file_path, 'r') as file:
    api_key = file.read().strip()

API_TOKEN = api_key
PLAYER_TAG = "#2LGY9G"  # Replace with the player's tag (include the #)

# Headers for the API request
HEADERS = {
    "Authorization": f"Bearer {API_TOKEN}"
}

# URL for player information
BASE_URL = "https://api.clashroyale.com/v1"
PLAYER_URL = f"{BASE_URL}/players/{PLAYER_TAG.replace('#', '%23')}"

def fetch_player_data():
    try:
        response = requests.get(PLAYER_URL, headers=HEADERS)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def save_to_csv(player_data):
    # Define the CSV filename
    filename = "player_stats.csv"

    # Extract relevant information
    player_name = player_data.get("name", "N/A")
    trophies = player_data.get("trophies", "N/A")
    clan = player_data.get("clan", {}).get("name", "No Clan")
    deck = [
        card["name"] for card in player_data.get("currentDeck", [])
    ]

    # Write to CSV
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Player Name", "Trophies", "Clan", "Deck"])
        writer.writerow([player_name, trophies, clan, ", ".join(deck)])
    
    print(f"Data saved to {filename}")

def main():
    player_data = fetch_player_data()
    if player_data:
        save_to_csv(player_data)

if __name__ == "__main__":
    main()


Data saved to player_stats.csv


In [35]:
with open('../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
project = hopsworks.login()
fs = project.get_feature_store() 

2025-01-07 23:30:18,291 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-07 23:30:18,302 INFO: Initializing external client
2025-01-07 23:30:18,304 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 23:30:19,891 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164438


In [36]:
# Retrieve feature groups for Clash Royale
player_stats_fg = fs.get_feature_group(
    name='clash_royale_onehotencoding',
    version=5,
)
game_events_fg = fs.get_feature_group(
    name='clash_royale_dataset_onehotencoding',
    version=5,
)


In [37]:

!pip install xgboost==1.6.2
!pip install scikit-learn==1.2.2



   ---------------------------------------- 0.0/125.4 MB ? eta -:--:--
   ---------------------------------------- 1.0/125.4 MB 6.3 MB/s eta 0:00:20
    --------------------------------------- 2.6/125.4 MB 6.6 MB/s eta 0:00:19
   - -------------------------------------- 3.9/125.4 MB 6.5 MB/s eta 0:00:19
   - -------------------------------------- 5.2/125.4 MB 6.6 MB/s eta 0:00:19
   -- ------------------------------------- 6.8/125.4 MB 6.7 MB/s eta 0:00:18
   -- ------------------------------------- 8.1/125.4 MB 6.6 MB/s eta 0:00:18
   --- ------------------------------------ 9.4/125.4 MB 6.6 MB/s eta 0:00:18
   --- ------------------------------------ 11.0/125.4 MB 6.6 MB/s eta 0:00:18
   --- ------------------------------------ 12.3/125.4 MB 6.7 MB/s eta 0:00:17
   ---- ----------------------------------- 13.6/125.4 MB 6.6 MB/s eta 0:00:17
   ---- ----------------------------------- 15.2/125.4 MB 6.6 MB/s eta 0:00:17
   ----- ---------------------------------- 16.5/125.4 MB 6.6 MB/s 

  You can safely remove it manually.


   ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
   ----- ---------------------------------- 1.0/8.3 MB 5.6 MB/s eta 0:00:02
   ------------ --------------------------- 2.6/8.3 MB 6.9 MB/s eta 0:00:01
   ------------------ --------------------- 3.9/8.3 MB 6.7 MB/s eta 0:00:01
   ------------------------- -------------- 5.2/8.3 MB 6.6 MB/s eta 0:00:01
   ------------------------------- -------- 6.6/8.3 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------  8.1/8.3 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 8.3/8.3 MB 6.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


  You can safely remove it manually.


In [38]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="clash_royale_xgboost_model",
    version=1,
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()
saved_model_dir

'C:\\Users\\jacop\\AppData\\Local\\Temp\\2bae5355-6f9a-42d5-a04b-ecb148acc242\\clash_royale_xgboost_model/1'

In [39]:
# Loading the XGBoost regressor model and label encoder from the saved model directory

retrieved_xgboost_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
#retrieved_xgboost_model

AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [ ]:
# QUA CE DA FARE IL DATAFRAME CON I DECK, deck 1 viene retrivato dall API , dentro player.csv, deck 2 viene scelto 

In [ ]:
df['result'] = retrieved_xgboost_model.predict(
    df[['deck1','deck2']])